# Implementation of ALAP Comparison Trials for C Invertible


In [51]:
using Test
using JuMP
using Gurobi
using LinearAlgebra

In [92]:
include("./mpc_functions.jl")
include("./mpc_functions_tests.jl")
include("./linear_system.jl")
include("./linear_system_test.jl")

Test Summary:                | Pass  Total
compute_Sx0() Function Tests |    1      1
[0.0; 1.0]
[0.0 0.0; 1.0 0.0; 2.0 1.0]
Test Summary:               | Pass  Total
compute_Sw() Function Tests |    4      4
Test Summary:                | Pass  Total
compute_C_M() Function Tests |    1      1
Test Summary:             | Pass  Total
LinearSystem Class Struct |    1      1
Test Summary: | Pass  Total
check() tests |    3      3
Test Summary: | Pass  Total
x_dim() Tests |    2      2
Test Summary: | Pass  Total
y_dim() Tests |    4      4
Test Summary:                       | Pass  Total
define_simple_eta_HPolytope() Tests |    2      2
Test Summary:                         | Pass  Total
find_ALAP_time_from_X0_to_bound Tests |    1      1
Academic license - for non-commercial use only - expires 2020-12-31
[0.0 0.0 0.0; 1.0 0.0 0.0; 1.0 1.0 0.0; 1.0 1.0 1.0]
find_est_error_bound_from_time_0_to_T Tests: Error During Test at /Users/kwesirutledge/Documents/Research/measurement-scheduling0/ju

LoadError: LoadError: Some tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.
in expression starting at /Users/kwesirutledge/Documents/Research/measurement-scheduling0/julia_code/linear_system_test.jl:109

In [91]:
A = [1 0;
     -0.5 2];
B = [1;0]
C = [0 1]
eta_w = 0.2
eta_v = 0.1

sys1 = LinearSystem(A,B,C,eta_w,eta_v)

T = 3

n_x = x_dim(sys1)
n_y = y_dim(sys1)

n_w = n_x
n_v = n_y

model = Model(Gurobi.Optimizer)

# compute H_w, H_v, and H_x0
H_w = [I(n_x);-I(n_x)]
h_w = sys1.eta_w*ones(2*n_x,1)
n_Hw = 2*n_w; #Dimensions of H_w A matrix

H_v = [I(n_y);-I(n_y)]
h_v = sys1.eta_v*ones(2*n_y,1)
n_Hv = 2*n_v; #Dimensions of H_v A matrix

eta_x0 = 0.1
H_x0 = [I(n_x);-I(n_x)]
h_x0 = eta_x0*ones(2*n_x,1)
n_Hx0 = 2*n_x; #Dimensions of H_x0 A matrix

# compute H_eta and h_eta
H_eta=[kron(Matrix(1I,T,T), H_w) zeros(T*n_Hw,T*n_y + n_x) ;
    zeros(T*n_Hv, T*n_w) kron(Matrix(1I,T,T), H_v) zeros(T*n_Hv,n_x);
    zeros(n_Hx0,T*(n_w+n_v)) H_x0]

h_eta=[kron(ones(T,1),h_w); kron(ones(T,1),h_v); h_x0];

#Clear the model?
empty!(model)

#Create the Optimization Variables
@variable(model,Q[1:n_x*T,1:n_y*T])
@variable(model,r[1:n_x*T,1])
@variable(model,Lambda[1:2*n_x*(T+1),1:T*(n_Hw+n_Hv)+n_Hx0])
@variable(model,alpha0[1:T+1,1])

#Constraints
@constraint(model,alpha0.>=0)
@constraint(model,Lambda.>=zeros(size(Lambda)))

S = compute_Sw( sys1.A , T )
C_bar = compute_C_M( sys1.C , [T-1] , T )
J = compute_Sx0( sys1.A , T)

# println(size(S))
# println(size(C_bar))
# println(size(J))
# println(size(Q))

println("Reach1")

P_xi_w = S + S*Q*C_bar*S
P_xi_v = S * Q 
P_xi_xi0 = ( I + S*Q*C_bar)*J

println("Reach2")

R_T_mT = [ I ; -Matrix(1I, n_x*(T+1), n_x*(T+1)) ]

# Add dual equality constraint
LHS1 = Lambda * H_eta
RHS1 = R_T_mT * [ P_xi_w P_xi_v P_xi_xi0 ]

@constraint(model, LHS1 .== RHS1 )

println("Reach3")

# Add dual inequality constraint
alpha_bar=kron(alpha0,ones(x_dim(sys1),1))
println(alpha_bar)
alpha_bar=[alpha_bar; alpha_bar];

size(Lambda)
size(h_eta)

LHS2 = Lambda * h_eta    
RHS2 = alpha_bar - R_T_mT * S*r
@constraint(model, LHS2.<=RHS2 )

println("Reach4")

# low diagonal constraint
for blk_row_idx = 1:div(size(Q,1), n_x)-1
    LHS_bloc = Q[(blk_row_idx-1)*n_x.+(1:n_x),blk_row_idx*n_y+1:end]
    @constraint(model,LHS_bloc.==zeros(size(LHS_bloc)))
end

#Create Objective
@objective(model, Min, sum(alpha0))

#Optimize!
optimize!(model)

print("termination_status = $(termination_status(model))\n")

# println(model)
println(objective_value(model))

Academic license - for non-commercial use only - expires 2020-12-31
Reach1
Reach2
Reach3
GenericAffExpr{Float64,VariableRef}[alpha0[1,1]; alpha0[1,1]; alpha0[2,1]; alpha0[2,1]; alpha0[3,1]; alpha0[3,1]; alpha0[4,1]; alpha0[4,1]]
Reach4
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 554 rows, 380 columns and 1352 nonzeros
Model fingerprint: 0x5a3ae2c0
Coefficient statistics:
  Matrix range     [1e-01, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 8e+00]
Presolve removed 542 rows and 355 columns
Presolve time: 0.00s
Presolved: 12 rows, 25 columns, 66 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1986000e+00   5.000700e+00   0.000000e+00      0s
       7    2.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.00 seconds
Optimal objective  2.500000000e+00

User

In [54]:
find_est_error_bound_from_time_0_to_T( sys1 , T , eta_x0 )

Academic license - for non-commercial use only - expires 2020-12-31
H_eta
  [1 ,  1]  =  1.0
  [3 ,  1]  =  -1.0
  [2 ,  2]  =  1.0
  [4 ,  2]  =  -1.0
  [5 ,  3]  =  1.0
  [7 ,  3]  =  -1.0
  [6 ,  4]  =  1.0
  [8 ,  4]  =  -1.0
  [9 ,  5]  =  1.0
  [11,  5]  =  -1.0
  [10,  6]  =  1.0
  [12,  6]  =  -1.0
  [13,  7]  =  1.0
  [14,  7]  =  -1.0
  [15,  8]  =  1.0
  [16,  8]  =  -1.0
  [17,  9]  =  1.0
  [18,  9]  =  -1.0
  [19, 10]  =  1.0
  [21, 10]  =  -1.0
  [20, 11]  =  1.0
  [22, 11]  =  -1.0
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 137 rows, 89 columns and 269 nonzeros
Model fingerprint: 0x4ea7a779
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 8e+00]
Presolve removed 137 rows and 89 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Ob

(MathOptInterface.OPTIMAL, 2.95)

In [80]:
print(size([1]))
temp1 = [1 0;0 2]
println(temp1)

temp1[1:1,2:2] .= 2
println(temp1)

println(typeof(1))
println(typeof([1]))
println(typeof([1 2;3 4]))

(1,)[1 0; 0 2]
[1 2; 0 2]
Int64
Array{Int64,1}
Array{Int64,2}
